In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config("spark.jars", "/data/cuspatial_data/jars/cudf-22.02.0-cuda11.jar,/data/cuspatial_data/jars/rapids-4-spark_2.12-22.02.0.jar,/data/cuspatial_data/jars/spark-cuspatial-22.06.0-SNAPSHOT.jar") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.executor.memory", "20GB") \
    .config("spark.executor.cores", "6") \
    .config("spark.plugins", "com.nvidia.spark.SQLPlugin") \
    .config("spark.executor.resource.gpu.amount", "1") \
    .getOrCreate()
# register the udf
spark.udf.registerJavaFunction("point_in_polygon", "com.nvidia.spark.rapids.udf.PointInPolygon", None)

In [ ]:
# prepare shape files
spark.sparkContext.addFile("/data/cuspatial_data/polygons/polygons.shp")
spark.sparkContext.addFile("/data/cuspatial_data/polygons/polygons.shx")

inputPath = "/data/cuspatial_data/points/"
outputPath = "/data/cuspatial_data/output/"

In [ ]:
# set its parameters via SQL config for runtime updating
spark.conf.set("spark.cuspatial.sql.udf.shapeFileName", "polygons.shp")

In [ ]:
# read the points data
df = spark.read.parquet(inputPath)
df = df.selectExpr('x', 'y', 'point_in_polygon(x, y) as point_in_polygon')

In [ ]:
import time
begin = time.time()
df.write.mode("overwrite").parquet(outputPath)
end = time.time()
print("==> It took {} s".format(round(end-begin, 2)))